# Evaluate Data Drift for train/test datasets

In [1]:
%load_ext autoreload
%autoreload 2

import joblib
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
import pandas as pd

from pathlib import Path
from sklearn import ensemble
from typing import Dict

from evidently.pipeline.column_mapping import ColumnMapping

from config import MLFLOW_TRACKING_URI, DATA_DIR, FILENAME, REPORTS_DIR

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Load Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [3]:
# Download original dataset with: python src/pipelines/load_data.py 

raw_data = pd.read_csv(f"../{DATA_DIR}/{FILENAME}")

raw_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Define column mapping

In [4]:
target = 'cnt'
prediction = 'prediction'
datetime = 'dteday'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]

column_mapping = ColumnMapping()
column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.datetime = datetime
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

# Define the comparison windows

In [5]:
start_date_0 = '2011-01-02 00:00:00'
end_date_0 = '2011-01-30 23:00:00'

experiment_batches = [
    
    ('2011-01-31 00:00:00','2011-02-06 23:00:00'),
    ('2011-02-07 23:00:00','2011-02-13 23:00:00'),
    ('2011-02-14 23:00:00','2011-02-20 23:00:00'),
    ('2011-02-21 00:00:00','2011-02-27 23:00:00'),
    ('2011-02-28 00:00:00','2011-03-06 23:00:00'),  
]

## Define the Reference data

In [6]:
# Set datetime index 
raw_data = raw_data.set_index('dteday')

# Define the reference dataset
reference = raw_data.loc[start_date_0:end_date_0]

print(reference.shape)
reference.head()

(617, 16)


,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
dteday,,,,,,,,,,,,,,,,
2011-01-03,48,1,0,1,0,0,1,1,1,0.22,0.1970,0.44,0.3582,0,5,5
2011-01-03,49,1,0,1,1,0,1,1,1,0.20,0.1667,0.44,0.4179,0,2,2
2011-01-03,50,1,0,1,4,0,1,1,1,0.16,0.1364,0.47,0.3881,0,1,1
2011-01-03,51,1,0,1,5,0,1,1,1,0.16,0.1364,0.47,0.2836,0,3,3
2011-01-03,52,1,0,1,6,0,1,1,1,0.14,0.1061,0.50,0.3881,0,30,30


## Train Models

In [7]:
from src.reports import (
    build_regression_quality_report,
    get_regression_quality_metrics,
    build_data_drift_report,
    get_data_drift_metrics,
)

In [8]:
# Set up MLFlow Client
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
print(f"Client tracking uri: {client.tracking_uri}")

# Set experiment name
mlflow.set_experiment("Data Drift")

# Set experiment variables
model_path = Path('../models/model.joblib')
ref_end_data = end_date_0

2023/08/14 13:21:36 INFO mlflow.tracking.fluent: Experiment with name 'Data Drift' does not exist. Creating a new experiment.


Client tracking uri: http://localhost:5000


In [9]:

ref_end_data = end_date_0
FEATURE_COLUMNS = numerical_features + categorical_features

# Start a new Run (Parent Run)
with mlflow.start_run() as run: 
    
    # Show newly created run metadata info
    print("Experiment id: {}".format(run.info.experiment_id))
    print("Run id: {}".format(run.info.run_id))
    print("Run name: {}".format(run.info.run_name))
    run_id = run.info.run_id
    
    # Save every fold metrics to a single object
    metrics_model = {}  # Model Quality metrics on train  (averaged)
    metrics_data = {}   # Data Drift metrics (averaged)

    # Run model train for each batch (K-Fold)
    for k, test_dates in enumerate(experiment_batches):
        
        print(f"Batch: {k}")
        
        train_dates = start_date_0, ref_end_data
        ref_end_data = test_dates[1] # Update reference end date for the next train batch 
        print(f"Train dates: {train_dates}") 
        print(f"Test (current) dates: {test_dates}") 
        
        train_data = raw_data.loc[train_dates[0]:train_dates[1]]
        X_train = train_data.loc[:, FEATURE_COLUMNS]
        y_train = train_data.loc[:, target]
        print("X_train (reference) dataset shape: ", X_train.shape, y_train.shape)
        
        test_data = raw_data.loc[test_dates[0]:test_dates[1]]
        X_test = test_data.loc[:, FEATURE_COLUMNS]
        y_test = test_data[target]
        print("X_test (current)) dataset shape: ",  X_test.shape, y_test.shape)
        
        # Train model
        regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)
        regressor.fit(X_train, y_train)
        
        # Make predictions 
        ref_prediction = regressor.predict(train_data[FEATURE_COLUMNS])
        train_data['prediction'] = ref_prediction
        cur_prediction = regressor.predict(test_data[FEATURE_COLUMNS])
        test_data['prediction'] = cur_prediction
        
        # Calculate Model Quality metrics
        regression_quality_report = build_regression_quality_report(
            reference_data=train_data, 
            current_data=test_data,
            column_mapping=column_mapping
        )
        
        # Extract Metrics from the  the Model Quality report
        train_metrics = get_regression_quality_metrics(regression_quality_report)
        metrics_model.update({test_dates[1]: train_metrics})

        # Calculate Data Drift metrics
        data_drift_report = build_data_drift_report(
            reference_data=X_train.reset_index(drop=True), 
            current_data=X_test.reset_index(drop=True),
            column_mapping=column_mapping,
            drift_share=0.4
        )
        data_drift_metrics: Dict = get_data_drift_metrics(data_drift_report)
        metrics_data.update({test_dates[1]: data_drift_metrics})
        
        # Run a Child Run for each Fold 
        with mlflow.start_run(run_name=test_dates[1], nested=True) as nested_run:
            
            # Show newly created run metadata info
            print("Run id: {}".format(nested_run.info.run_id))
            print("Run name: {}".format(nested_run.info.run_name))

            # Log parameters
            mlflow.log_param("begin", test_dates[0])
            mlflow.log_param("end", test_dates[1])
            
            # Log metrics
            mlflow.log_metrics(train_metrics)
            mlflow.log_metrics(data_drift_metrics)
            
            # Save the Model Quality report & Log  
            model_quality_report_path = f"../{REPORTS_DIR}/model_quality_report.html"
            regression_quality_report.save_html(model_quality_report_path)
            mlflow.log_artifact(model_quality_report_path)
            
            # Log Data Drift report ONLY if drift is detected
            if data_drift_metrics['dataset_drift'] is True:
                report_date = test_dates[1].split(' ')[0]
                data_drift_report_path = f"../{REPORTS_DIR}/data_drift_report_{report_date}.html"
                data_drift_report.save_html(data_drift_report_path)
                mlflow.log_artifact(data_drift_report_path)
    
    # Save final model
    joblib.dump(regressor, model_path)
    
    # Log the last batch model as the parent Run model
    mlflow.log_artifact(model_path)
    
    # Log metrics
    avg_model_metrics = pd.DataFrame.from_dict(metrics_model).T.mean().round(3).to_dict()
    mlflow.log_metrics(avg_model_metrics)
    
    avg_data_metrics = pd.DataFrame.from_dict(metrics_data).T.mean().round(3).to_dict()
    mlflow.log_metrics(avg_data_metrics)

Experiment id: 501037242071821375
Run id: 8aea2cf44c8e4aad8a4e606560dbf8de
Run name: beautiful-fox-682
Batch: 0
Train dates: ('2011-01-02 00:00:00', '2011-01-30 23:00:00')
Test (current) dates: ('2011-01-31 00:00:00', '2011-02-06 23:00:00')
X_train (reference) dataset shape:  (617, 10) (617,)
X_test (current)) dataset shape:  (141, 10) (141,)
Run id: 9f35f9bd62b54f5aa853fbd341d257a4
Run name: 2011-02-06 23:00:00
Batch: 1
Train dates: ('2011-01-02 00:00:00', '2011-02-06 23:00:00')
Test (current) dates: ('2011-02-07 23:00:00', '2011-02-13 23:00:00')
X_train (reference) dataset shape:  (782, 10) (782,)
X_test (current)) dataset shape:  (139, 10) (139,)
Run id: 26a93a6a2e4843f3b734806d9e94cf5e
Run name: 2011-02-13 23:00:00
Batch: 2
Train dates: ('2011-01-02 00:00:00', '2011-02-13 23:00:00')
Test (current) dates: ('2011-02-14 23:00:00', '2011-02-20 23:00:00')
X_train (reference) dataset shape:  (945, 10) (945,)
X_test (current)) dataset shape:  (141, 10) (141,)
Run id: 87b860de9a5547b29273c